Step 1: Simulate the Dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

np.random.seed(42)

# --- Parameters
members = [f"MEM{i:04}" for i in range(200)]
trainers = ['Trainer A', 'Trainer B', 'Trainer C']
genders = ['Male', 'Female', 'Other']

member_data = []
checkins_data = []

for member in members:
    join_date = datetime(2023, random.randint(1, 12), random.randint(1, 28))
    retention_days = np.random.randint(30, 365)
    cancel_chance = np.random.rand()
    cancel_date = join_date + timedelta(days=retention_days) if cancel_chance < 0.4 else pd.NaT
    age = np.random.randint(18, 60)
    gender = np.random.choice(genders)
    trainer = np.random.choice(trainers)

    member_data.append({
        'Member ID': member,
        'Gender': gender,
        'Age': age,
        'Join Date': join_date,
        'Cancel Date': cancel_date,
        'Trainer Assigned': trainer
    })

    # Generate check-ins (approx 2–5/week until cancel date or today)
    end_date = cancel_date if pd.notnull(cancel_date) else datetime(2024, 12, 31)
    days_range = pd.date_range(start=join_date, end=end_date, freq='D')
    checkin_days = np.random.choice(days_range, size=np.random.randint(40, 150))

    for checkin in checkin_days:
        checkins_data.append({
            'Member ID': member,
            'Check-in Date': checkin
        })

members_df = pd.DataFrame(member_data)
checkins_df = pd.DataFrame(checkins_data)
members_df.to_csv('gym_members.csv', index=False)
checkins_df.to_csv('gym_checkins.csv', index=False)


Step 2: Load and Prepare Data

In [2]:
members_df = pd.read_csv('gym_members.csv', parse_dates=['Join Date', 'Cancel Date'])
checkins_df = pd.read_csv('gym_checkins.csv', parse_dates=['Check-in Date'])


Step 3: Retention Analysis – Who Quits and When?

In [3]:
members_df['Cancelled'] = members_df['Cancel Date'].notna()
members_df['Retention Days'] = (members_df['Cancel Date'].fillna(pd.Timestamp('2024-12-31')) - members_df['Join Date']).dt.days

# Group by age groups or gender
retention_by_gender = members_df.groupby('Gender')['Retention Days'].mean().reset_index()


Plot: Avg Retention by Gender

In [4]:
import plotly.express as px

fig1 = px.bar(
    retention_by_gender,
    x='Gender',
    y='Retention Days',
    title='Average Retention Days by Gender',
    color='Retention Days',
    text='Retention Days'
)
fig1.show()


 Step 4: Peak Gym Hours Analysis

In [5]:
checkins_df['Hour'] = checkins_df['Check-in Date'].dt.hour
checkins_df['Weekday'] = checkins_df['Check-in Date'].dt.day_name()

# Group by hour
hourly_checkins = checkins_df.groupby('Hour').size().reset_index(name='Check-ins')


Plot: Peak Check-in Hours

In [6]:
fig2 = px.bar(
    hourly_checkins,
    x='Hour',
    y='Check-ins',
    title='Peak Gym Hours by Check-in Frequency',
    text='Check-ins'
)
fig2.show()


Step 5: Heatmap – Weekday vs Hour Check-ins

In [7]:
heatmap = checkins_df.groupby(['Weekday', 'Hour']).size().reset_index(name='Count')
heatmap = heatmap.pivot(index='Weekday', columns='Hour', values='Count').fillna(0)
heatmap = heatmap.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

import plotly.graph_objects as go

fig3 = go.Figure(
    data=go.Heatmap(
        z=heatmap.values,
        x=heatmap.columns,
        y=heatmap.index,
        colorscale='Viridis'
    )
)
fig3.update_layout(title='Check-in Heatmap: Day of Week vs Hour')
fig3.show()


 Step 6: Trainer Effectiveness
 
Calculate:
Avg retention per trainer

Total check-ins per trainer’s members

In [8]:
# Merge to get trainer per check-in
checkins_with_trainer = checkins_df.merge(members_df[['Member ID', 'Trainer Assigned']], on='Member ID')

trainer_checkins = checkins_with_trainer.groupby('Trainer Assigned').size().reset_index(name='Total Check-ins')
trainer_retention = members_df.groupby('Trainer Assigned')['Retention Days'].mean().reset_index()

trainer_summary = pd.merge(trainer_checkins, trainer_retention, on='Trainer Assigned')


Plot: Trainer Performance

In [9]:
fig4 = px.bar(
    trainer_summary,
    x='Trainer Assigned',
    y='Total Check-ins',
    color='Retention Days',
    title='Trainer Effectiveness: Check-ins vs Retention',
    text='Retention Days'
)
fig4.show()


Insights You Can Offer
Who quits early? → Target by gender, age group

Peak gym usage → Helps optimize staff and equipment allocation

Trainer impact → Retention and engagement tied to trainers

Weekly/hourly heatmap → Adjust hours/classes based on peak demand

Exporting All Plots To Single HTML File

In [10]:
import plotly.io as pio

# List of all your figures
figures = [fig1, fig2, fig3, fig4]

# Generate HTML strings for each figure
html_parts = [pio.to_html(fig, full_html=False, include_plotlyjs='cdn') for fig in figures]

# Combine them into one full HTML document
full_html = f"""
<html>
<head>
    <title>Spotify Analysis</title>
</head>
<body>
    {''.join(html_parts)}
</body>
</html>
"""

# Save to file
output_path = "Gym Member Retention & Attendance Analysis.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"Saved to {output_path}")

Saved to Gym Member Retention & Attendance Analysis.html
